In [114]:
import sys
import string
import simplejson
from twython import Twython
import datetime

import numpy as np
import pandas as pd
%pylab inline
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split


Populating the interactive namespace from numpy and matplotlib


/Users/lanimc/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Starting Dataset: NYC Social Media Usage<br>
Twitter and Facebook statistics from various NYC agencies and organizations.<br>
NYC OpenData<br>
created Oct 11, 2011<br>
updated Nov 29, 2016

In [115]:
data = pd.read_json('https://data.cityofnewyork.us/resource/pbc3-75xt.json')  

Identify top social media users and categories

In [116]:
data_agg = data.groupby(['agency','platform']).agg({'metric':sum})

In [117]:
data_agg.head(5)

metric
agency platform            
311    Facebook      6991.0
       Twitter     133435.0
       WordPress        NaN
       iPhone App   70041.0
ACS    Facebook      2020.0

Use Twitter API to grab user information from list of top twitter posters from 2011-2012 data; Aim is to get a good crossection of types of tweets agencies send out - events, notifications, updates, press, service response, requests for assistance, customer service etc.

export text file

Uses Twython module to access Twitter API

Method from http://social-metrics.org/twitter-user-data/

In [118]:
now = datetime.datetime.now()
day=int(now.day)
month=int(now.month)
year=int(now.year)

t = Twython(app_key='Xt4YwW3pSY4Md2w9RbwqR8wXO', 
    app_secret='cyQHNkZpanA0TG7XqVgxmxrGvyIZYdYVcfrauFgoi6eQ2ONGP3',
    oauth_token='484334783-yDabOpGySqX1CAKVhAECUNWdsTGguY8Cp74lkSk8',
    oauth_token_secret='RTSxb5vtYnLpmDG3bbO6MCtwXLgVzg3TQhXALA2JOTJ3D')

ids = "NYCMayorsOffice,nycgov, NYPDnews, FDNY, NYCSchools, NYCParks,nycoem, NYCHA,nyc311"

users = t.lookup_user(screen_name = ids)

outfn = "twitter_user_data_%i.%i.%i.txt" % (now.month, now.day, now.year)

fields = "id screen_name name created_at url followers_count friends_count statuses_count \
    favourites_count listed_count \
    contributors_enabled description protected location lang expanded_url".split()
 
outfp = open(outfn, "w")
outfp.write(string.join(fields, "\t") + "\n")  # header

for entry in users:
    #CREATE EMPTY DICTIONARY
    r = {}
    for f in fields:
        r[f] = ""
    #ASSIGN VALUE OF 'ID' FIELD IN JSON TO 'ID' FIELD IN PYTHON DICTIONARY
    r['id'] = entry['id']
    r['screen_name'] = entry['screen_name']
    r['name'] = entry['name']
    r['created_at'] = entry['created_at']
    r['url'] = entry['url']
    r['followers_count'] = entry['followers_count']
    r['friends_count'] = entry['friends_count']
    r['statuses_count'] = entry['statuses_count']
    r['favourites_count'] = entry['favourites_count']
    r['listed_count'] = entry['listed_count']
    r['contributors_enabled'] = entry['contributors_enabled']
    r['description'] = entry['description']
    r['protected'] = entry['protected']
    r['location'] = entry['location']
    r['lang'] = entry['lang']
    #NOT EVERY ID WILL HAVE A 'URL' KEY, SO CHECK FOR ITS EXISTENCE WITH IF CLAUSE
    if 'url' in entry['entities']:
        r['expanded_url'] = entry['entities']['url']['urls'][0]['expanded_url']
    else:
        r['expanded_url'] = ''
    print r
    #CREATE EMPTY LIST
    lst = []
    #ADD DATA FOR EACH VARIABLE
    for f in fields:
        lst.append(unicode(r[f]).replace("\/", "/"))
    #WRITE ROW WITH DATA IN LIST
    outfp.write(string.join(lst, "\t").encode("utf-8") + "\n")

outfp.close()   

{'lang': u'en', 'statuses_count': 16874, 'name': u"NYC Mayor's Office", 'friends_count': 609, 'url': u'https://t.co/gbCFBEvClg', 'created_at': u'Thu Jul 09 19:42:27 +0000 2009', 'description': u'Live from City Hall, in the greatest city on earth. @NYCMayor Bill de Blasio.', 'contributors_enabled': False, 'followers_count': 578639, 'protected': False, 'expanded_url': u'http://www.NYC.gov', 'location': u'New York City', 'favourites_count': 530, 'listed_count': 5376, 'id': 55338739, 'screen_name': u'NYCMayorsOffice'}
{'lang': u'en', 'statuses_count': 18545, 'name': u'City of New York', 'friends_count': 202, 'url': u'https://t.co/HSvs3qcfig', 'created_at': u'Sat Feb 12 00:43:55 +0000 2011', 'description': u'Official New York City government Twitter. Keep up with NYC news, services, programs, free events and emergency notifications https://t.co/N0igPl3T7S', 'contributors_enabled': False, 'followers_count': 712593, 'protected': False, 'expanded_url': u'http://nyc.gov', 'location': u'New York

In [119]:
twittersummarydata = pd.read_table("twitter_user_data_%i.%i.%i.txt" % (now.month, now.day, now.year))  

In [120]:
twittersummarydata.head(3)

,id,screen_name,name,created_at,url,followers_count,friends_count,statuses_count,favourites_count,listed_count,contributors_enabled,description,protected,location,lang,expanded_url
0,55338739,NYCMayorsOffice,NYC Mayor's Office,Thu Jul 09 19:42:27 +0000 2009,https://t.co/gbCFBEvClg,578639,609,16874,530,5376,False,"Live from City Hall, in the greatest city on e...",False,New York City,en,http://www.NYC.gov
1,250884927,nycgov,City of New York,Sat Feb 12 00:43:55 +0000 2011,https://t.co/HSvs3qcfig,712593,202,18545,576,3919,False,Official New York City government Twitter. Kee...,False,New York City,en,http://nyc.gov
2,17393196,NYPDnews,NYPD NEWS,Fri Nov 14 19:00:10 +0000 2008,https://t.co/bEB0pCXJXR,267641,280,13535,1456,3540,False,The official Twitter of the New York City Poli...,False,New York City,en,http://www.nyc.gov/nypd


In [121]:
twittersummarydata.describe()

,id,followers_count,friends_count,statuses_count,favourites_count,listed_count
count,9.000000e+00,9.000000,9.000000,9.000000,9.000000,9.000000
mean,1.001523e+08,250405.111111,633.333333,16000.333333,1248.555556,2691.888889
std,8.971664e+07,239492.156794,630.300920,6353.996184,1034.928393,1483.682197
min,1.739320e+07,19129.000000,202.000000,6645.000000,53.000000,548.000000
25%,3.003155e+07,97331.000000,270.000000,11651.000000,530.000000,1904.000000
50%,5.533874e+07,180347.000000,280.000000,15570.000000,1109.000000,2533.000000
75%,1.348466e+08,267641.000000,609.000000,18545.000000,1789.000000,3540.000000
max,2.508849e+08,712593.000000,1954.000000,26439.000000,3335.000000,5376.000000


#get tweets from these agencies<br>
in order to sve time, I used Twlet.com to get 3200 most recent tweets from 8 city agencies. <br> downloaded as excel files.